# Análisis de datos lingüisticos

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ivanvladimir/aProyectos-MeIA/blob/main/Programacion1.ipynb)
[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ivanvladimir/Proyectos-MeIA/blob/main/Programacion1.ipynb)

Este es el código para ejemplificar análisis computacional lingüístico.

### Instrucciones

Ejecutar las celdas en el orden que se encuentran.

### Licencia de la notebook

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.

### Información general

> **Author(s)**: <a href="https://twitter.com/ivanvladimir">@ivanvladimir</a> </br>
> **Last updated**: 15/06/2025

# ❶  Preparar librerias 

In [8]:
# Cargar librerias

import nltk
import pandas as pd
import os

In [63]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/ivanvladimir/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ivanvladimir/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# ❷ Preparar datos 

In [3]:
# Bajar datos mañanera
!git clone https://github.com/NOSTRODATA/conferencias_matutinas_amlo.git

Cloning into 'conferencias_matutinas_amlo'...
remote: Enumerating objects: 34386, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 34386 (delta 96), reused 281 (delta 75), pack-reused 34081 (from 2)
Receiving objects: 100% (34386/34386), 1.21 GiB | 21.66 MiB/s, done.
Resolving deltas: 100% (1906/1906), done.
Updating files: 100% (19228/19228), done.


In [13]:
# Poner todos los datos en un dataframe

dataframes=[]

for root, dirs, files in os.walk("conferencias_matutinas_amlo/", topdown=False):
   for name in files:
      if name.startswith('mananera') and name.endswith(".csv"):
        try:
            filename=os.path.join(root,name)
            df = pd.read_csv(filename)
            df['source_file'] = filename
            dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {str(e)}")
            continue

try:
    df = pd.concat(dataframes, ignore_index=True, sort=False)
except Exception as e:
    print(f"Error combining dataframes: {str(e)}")


/tmp/ipykernel_1292894/1768951804.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dataframes, ignore_index=True, sort=False)


In [14]:
df

,Participante,Texto,Sentimiento,Palabras,Dia,Mes,Anio,source_file
0,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,: Buenos días. El día de hoy vamos a presentar...,0.022805,26,27,12,2018,conferencias_matutinas_amlo/2018/12-2018/dicie...
1,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Consideramos que se trata de un plan estratégi...,0.017787,38,27,12,2018,conferencias_matutinas_amlo/2018/12-2018/dicie...
2,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Es un robo que el año pasado significó, en tér...",0.026243,23,27,12,2018,conferencias_matutinas_amlo/2018/12-2018/dicie...
3,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,El año pasado se robaron diariamente más de 60...,0.106680,45,27,12,2018,conferencias_matutinas_amlo/2018/12-2018/dicie...
4,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Si pensamos en 600 pipas diarias, no estamos s...",0.006258,52,27,12,2018,conferencias_matutinas_amlo/2018/12-2018/dicie...
...,...,...,...,...,...,...,...,...
432646,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Miren, tenemos el apoyo de los ingenieros mili...",0.000000,79,11,3,2022,conferencias_matutinas_amlo/2022/3-2022/marzo ...
432647,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Es un gran aeropuerto, de los mejores del mund...",0.000000,59,11,3,2022,conferencias_matutinas_amlo/2022/3-2022/marzo ...
432648,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Entonces, yo confío mucho en los ingenieros mi...",0.000000,23,11,3,2022,conferencias_matutinas_amlo/2022/3-2022/marzo ...
432649,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Y nos vamos a seguir encontrando, porque yo vi...",0.000000,71,11,3,2022,conferencias_matutinas_amlo/2022/3-2022/marzo ...


# ❸ Calcular concordancias

In [46]:
from nltk.text import Text, ConcordanceIndex
from nltk.tokenize import word_tokenize

text = '\n'.join(df[df['Participante']=='PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR']['Texto'].astype(str))

tokens = word_tokenize(text)
concordance_index = ConcordanceIndex(tokens)

In [59]:
offsets = concordance_index.offsets("Pemex")
width=60
concordances = []

for offset in offsets:
    # Calculate context boundaries
    left_start = max(0, offset - width)
    right_end = min(len(tokens), offset + width + 1)
    
    # Extract contexts
    left_context = tokens[left_start:offset]
    keyword = tokens[offset]
    right_context = tokens[offset + 1:right_end]
    
    concordances.append({
        'position': offset,
        'left_context': left_context,
        'keyword': keyword,
        'right_context': right_context,
        'left_text': ' '.join(left_context),
        'right_text': ' '.join(right_context),
        'full_context': ' '.join(tokens[left_start:right_end])
    })

print(f"Total de concordancias {len(concordances)}")

Total de concordancias 2929


In [61]:
INI=0
FIN=20

for conc in concordances[INI:FIN]:
    print(f"{conc['left_text'][-width:]:>60} [{conc['keyword']}] {conc['right_text'][:width]:<60}")

no de la República para combatir el robo de hidrocarburos en [Pemex] . Consideramos que se trata de un plan estratégico necesario
 pasado significó , en términos generales , una pérdida para [Pemex] del orden de 60 mil millones de pesos . El año pasado se rob
 combatir este robo afuera y adentro de las instalaciones de [Pemex] . El director de Pemex , Octavio Romero Oropeza , va a infor
era y adentro de las instalaciones de Pemex . El director de [Pemex] , Octavio Romero Oropeza , va a informarles , lo mismo que e
bar con esta corrupción . Y yo convoco a los trabajadores de [Pemex] para que nos apoyen , nos ayuden a que se logre el propósito
 . : Hay información de que en algunas áreas no podía entrar [Pemex] , la autoridad , porque eran áreas reservadas o estaban cond
ió . Nada más se les informó a los dirigentes sindicales que [Pemex] , y no sólo Pemex , el gobierno federal y las Fuerzas Armada
es informó a los dirigentes sindicales que Pemex , y no sólo [Pemex] , el gobierno federal

# ❹ Calcular colocaciones

In [66]:
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures
from nltk.corpus import stopwords
import string

min_freq=2
num_collocations=20

# Remove stopwords and punctuation
stop_words = set(stopwords.words('spanish'))
tokens = [token for token in tokens 
          if token not in stop_words and token not in string.punctuation]

# Bigram collocations
bigram_finder = BigramCollocationFinder.from_words(tokens)
bigram_finder.apply_freq_filter(min_freq)

# Different scoring methods
print("=== BIGRAM COLLOCATIONS ===")

print(f"\nTop {num_collocations} by PMI (Pointwise Mutual Information):")
pmi_bigrams = bigram_finder.nbest(BigramAssocMeasures.pmi, num_collocations)
for bigram in pmi_bigrams:
    print(f"  {bigram[0]} {bigram[1]}")

print(f"\nTop {num_collocations} by Chi-square:")
chi_sq_bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, num_collocations)
for bigram in chi_sq_bigrams:
    print(f"  {bigram[0]} {bigram[1]}")

print(f"\nTop {num_collocations} by Likelihood Ratio:")
likelihood_bigrams = bigram_finder.nbest(BigramAssocMeasures.likelihood_ratio, num_collocations)
for bigram in likelihood_bigrams:
    print(f"  {bigram[0]} {bigram[1]}")

# Trigram collocations
trigram_finder = TrigramCollocationFinder.from_words(tokens)
trigram_finder.apply_freq_filter(min_freq)

print(f"\n=== TRIGRAM COLLOCATIONS ===")
print(f"\nTop {num_collocations//2} by PMI:")
pmi_trigrams = trigram_finder.nbest(TrigramAssocMeasures.pmi, num_collocations//2)
for trigram in pmi_trigrams:
    print(f"  {trigram[0]} {trigram[1]} {trigram[2]}")

=== BIGRAM COLLOCATIONS ===

Top 20 by PMI (Pointwise Mutual Information):
  -Villa lloraba-
  -contrario piensa-
  AZUCENA URESTI
  Arcadio Zentella
  Arnold Alondra
  BAJA CALIFORNIA
  BEATRIZ PAGÉS
  Boy Scouts
  Bringas Burelo
  CELORIO ALCÁNTARA
  CORREDOR INTEROCEÁNICO
  Caldos Zenón
  Cenote Aerolito
  DERECHOS HUMANOS
  Dina Boluarte
  ELECTRÓNICA INAOE
  EMMANUEL SIBILLA
  ESTEBAN VILLEGAS
  Emulsión Scott
  FEDERACIÓN PFF

Top 20 by Chi-square:
  -Villa lloraba-
  -contrario piensa-
  ADÁN AUGUSTO
  AL TURISMO
  ALAZRAKI GROSSMANN
  ALICIA BÁRCENA
  ARIADNA MONTIEL
  ATiempo.com.mx PuenteLibre.mx
  AZUCENA URESTI
  Aguilares Camines
  Arcadio Zentella
  Arnold Alondra
  BAJA CALIFORNIA
  BATRES GUADARRAMA
  BEATRIZ PAGÉS
  BLANCO BRAVO
  Bad Bunny
  Bebidas Alimentos
  Boy Scouts
  Bringas Burelo

Top 20 by Likelihood Ratio:
  Estados Unidos
  A ver
  Poder Judicial
  mil millones
  millones pesos
  Guardia Nacional
  Comisión Federal
  Federal Electricidad
  Tren Maya
  adul